In [54]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/bsebse/BSE102_Lecture 3_Chemical basis of life.pdf
/kaggle/input/aaaaaaaaaaaaaaaaaaaaaaa/BSE102_Lecture 3_Chemical basis of life.pdf


In [55]:
pip install pymupdf

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [56]:
pip install einops flash_attn

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [57]:
pip install python-docx

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [58]:
pip install langchain

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [59]:
pip install -U langchain-community

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [60]:
# pip install flash_attn einops

In [61]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.chains.summarize import load_summarize_chain
from transformers import T5Tokenizer, T5ForConditionalGeneration
from transformers import pipeline
import base64

In [62]:
import os
import fitz  
from PIL import Image, ImageDraw, ImageFont
import io
from io import BytesIO
import pytesseract  
import torch 
import requests  
from transformers import Blip2Processor, Blip2ForConditionalGeneration, PegasusForConditionalGeneration, PegasusTokenizer, AutoProcessor, AutoModelForCausalLM 
from docx import Document 
from docx.shared import Inches  
import tempfile  
from transformers import T5Tokenizer, T5ForConditionalGeneration
from transformers import BartForConditionalGeneration, BartTokenizer
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from transformers import BartTokenizer
import numpy as np 
import pandas as pd 
nltk.download('punkt')
nltk.download('stopwords')


[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [63]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [64]:
model = AutoModelForCausalLM.from_pretrained("microsoft/Florence-2-large", trust_remote_code=True).to(device)
processor = AutoProcessor.from_pretrained("microsoft/Florence-2-large", trust_remote_code=True)

In [65]:
def run_example(image, task_prompt, text_input=None):
    if text_input is None:
        prompt = task_prompt
    else:
        prompt = task_prompt + text_input
    inputs = processor(text=prompt, images=image, return_tensors="pt").to(device)
    generated_ids = model.generate(
        input_ids=inputs["input_ids"],
        pixel_values=inputs["pixel_values"],
        max_new_tokens=1024,
        num_beams=3
    )
    generated_text = processor.batch_decode(generated_ids, skip_special_tokens=False)[0]
    parsed_answer = processor.post_process_generation(generated_text, task=task_prompt, image_size=(image.width, image.height))
    caption = parsed_answer["<MORE_DETAILED_CAPTION>"]
    return caption

In [76]:
from PIL import Image
import io
import fitz

def pdf_doc(pdf_document):
    doc = fitz.open(pdf_document)

    page_text_list = []
    for page_number in range(len(doc)):
        page = doc.load_page(page_number)  
        image_list = page.get_images(full=True)  
        page_text = page.get_text()

        # Process each image on the page
        for img_index, img in enumerate(image_list):
            xref = img[0] 
            base_image = doc.extract_image(xref)  
            image_bytes = base_image["image"]
            
            image = Image.open(io.BytesIO(image_bytes)).convert('RGB')
            
            prompt = "<MORE_DETAILED_CAPTION>"
            generated_caption = run_example(image, prompt)

            updated_paragraphs = []
            paragraphs = page_text.split('\n\n') 
            for para in paragraphs:
                updated_paragraphs.append(para)
                if para.strip():  
                    updated_paragraphs.append(f"Caption: {generated_caption}")

            page_text_combined = '\n\n'.join(updated_paragraphs)

        page_text_list.append(page_text_combined)
    return page_text_list


In [68]:
def docx_doc(docx_document):
    doc = Document(docx_document)
    doc_text_list = []
    for para in doc.paragraphs:
        doc_text_list.append(para.text)

        for run in para.runs:
            if run.element.xml.find("pic:blipFill") != -1:
                image = run.element.xpath(".//a:blip")[0]
                image_data = image.get("{http://schemas.openxmlformats.org/officeDocument/2006/relationships}embed")
                image_part = doc.part.related_parts[image_data]
                image_bytes = image_part.blob

                image = Image.open(io.BytesIO(image_bytes)).convert('RGB')
                prompt = "<MORE_DETAILED_CAPTION>"
                generated_caption = run_example(image, prompt)
                combined_text = f"Caption: {generated_caption}\n"
                doc_text_list.append(combined_text)
    return doc_text_list

    



In [69]:
def determine_file_type(file_path):
    _, file_extension = os.path.splitext(file_path)
    if file_extension.lower() == '.docx':
        return 'docx'
    elif file_extension.lower() == '.pdf':
        return 'pdf'
    else:
        return 'unknown'

In [70]:
def get_text(path):
    text_each_page=[]
    file_type=determine_file_type(path)
    if file_type=='pdf':
        text_each_page=pdf_doc(path)
    elif file_type=='docx':
        text_each_page=docx_doc(path)
    else :
        print("invalid file type")
    return text_each_page

In [71]:
def preprocess_text(text):
    text = text.lower()
    tokens = word_tokenize(text)
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word.isalpha() and word not in stop_words]
    return ' '.join(tokens)


In [72]:
path="/kaggle/input/aaaaaaaaaaaaaaaaaaaaaaa/BSE102_Lecture 3_Chemical basis of life.pdf"


In [73]:
# def process_text_in_sentence_chunks(path, max_chunk_size):
#     text=('\n'.join(page for page in get_text(path)))
#     sentences = text.replace("\n", " ") 
#     sentences = nltk.sent_tokenize(sentences)

#     current_chunk = ""
#     text_list = []
#     for sentence in sentences:
#         if len(current_chunk) + len(sentence) <= max_chunk_size:
#             current_chunk += " " + sentence
#         else:
#               text_list.append(current_chunk.strip())
#               current_chunk = sentence

#     if current_chunk:
#         text_list.append(current_chunk.strip())
#     return text_list

In [74]:
def file_preprocessing(file):
    pages=('\n'.join(page for page in get_text(path)))
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=50)
    texts = text_splitter.split_text(pages)
    final_texts = ""
    for text in texts:
        final_texts = final_texts + text
    return final_texts

In [77]:
k=file_preprocessing(path)

In [78]:
print(k)

BSE 102 Bioscience
Spring 2024
The Chemical Basis of Life
Instructor: Dr. Sunil Kumar Boda
Assistant Professor
Department of Biosciences & Biomedical Engineering
IIT IndoreIIT Indore
E-mail: sunilboda@iiti.ac.inCaption:The image is the logo of the Indian Institute of Technology Indore. It is a circular emblem with a blue background and a white border. In the center of the emblem, there is an open book with aof the emblem, there is an open book with a sunburst design around it. The book is surrounded by a banner with the words "IIT Indore" written in Hindi. Below the book, there are two lines of text inBelow the book, there are two lines of text in Hindi, which translates to "Indian Institute of technology Indore". The text below the book reads "II संस्थान सेवारीय साथिकाली" (Indian Institute forसंस्थान सेवारीय साथिकाली" (Indian Institute for Technology and Technology) in English. The emblem is a symbol of knowledge and knowledge in India.Organisms are composed of elements, usually combi